## Analysis of Gravitational Wave data from LIGO

Today we are going to follow the analysis of the LIGO data showing the first detection of gravitational waves.  You can find more information here:  https://losc.ligo.org/events/GW150914/   This notebook is based on one produced by the LIGO Open Science Center, https://losc.ligo.org/about/ , which has a variety of resources for learning about analyzing LIGO data. 

Our first step is to download the two datafiles at these links:

https://losc.ligo.org/s/events/GW150914/H-H1_LOSC_4_V1-1126259446-32.hdf5

https://losc.ligo.org/s/events/GW150914/L-L1_LOSC_4_V1-1126259446-32.hdf5

Save those files in the same directory as this notebook. 

The names of the files tell us about the data:

* "H-H1" means that the data come from the LIGO Hanford Observatory site and the LIGO "H1" datector;

* the "4" means the strain time-series data are (down-)sampled from 16384 Hz to 4096 Hz.  There are also corresponding versions that haven't been binned, which have a "32" in place of the "4" there. 

* the "V1" means version 1 of this data release;

* "1126257414-4096" means the data starts at GPS time 1126257414 (Mon Sep 14 09:16:37 GMT 2015), duration 4096 seconds; NOTE: GPS time is number of seconds since Jan 6, 1980 GMT. See http://www.oc.nps.edu/oc2902w/gps/timsys.html or https://losc.ligo.org/gps/

* the filetype "hdf5" means the data are in hdf5 format: https://www.hdfgroup.org/HDF5/

Helpfully, the LIGO team has also released a set of python functions to read the LIGO data.  However, they were written for Python 2 only, and some of you are running Python 3.  If you have Python 2 (look in the upper-right-hand corner of this notebook) then you can grab the LIGO original version: 

https://losc.ligo.org/s/sample_code/readligo.py

If you're running Python 3, this version should work:

https://www.dropbox.com/s/ba4nxbxspobn7jy/readligo.py?dl=0

Either way, save the file as 'readligo.py'.  You'll then be able to load this set of functions in just the same way as you have in the past for `numpy` or `scipy`, using `import`.

Our next step is to read in the data using a function in `readligo` called `loaddata`, which returns the strain data, the time data, and a dictionary file of data quality channels.  Dictionaries are a type of python data, where instead of integer indices like arrays, the data in a dictionary can be accessed by key, which can be strings, numbers, or tuples.


#### Exercise 1

Import the `readligo` functions and use `loaddata` to load the data from H1 and L1 downsampled to 4096 Hz (which are the first two data files listed above).  Note that you'll need to look at the code in `readligo.py` to see what input parameters that function takes, and what it returns (so that you can "catch" the return values appropriately in different variables; remember that a function can return more than one thing, so you want to have an appropriate number of variables on the left-hand side of the function call). 


Also, use the numpy function `genfromtxt` that we used last week to load a waveform generated from a theoretical numerical relativity template, found here,

https://losc.ligo.org/s/events/GW150914/GW150914_4_NR_waveform.txt

Plot the data in a window $\pm $5 s around the data.  Use masking as we have before to select the values in this range.  The event we're interested occurred at $t=$1126259462.422. Plot both L1 and H1 together, in different colors, and label your axes appropriately. 

You can't see the signal in this data.  We'll need to do some transformations on the data, but first let's see what the data looks like in the frequency domain.  Instead of doing the FFT, use matplotlib.mlab.psd(), a function that will both generate the frequency array and the power spectral density array so that you can plot them.  Plot the power spectral density of both H1 and L1 between 10 and 2000 Hz.  Below 10 Hz, LIGO is not properly calibrated, above 2000 Hz is data above the Nyquist frequency, which is half the sampling rate - is the highest frequency that can be coded at a given sampling rate in order to be able to fully reconstruct the signal.

The sharp spectral lines you see are from the LIGO instruments, for instance, the peak at about 500 Hz and harmonics are the mirror suspension resonance.

We can see that the data are very strongly "colored" - noise fluctuations are much larger at low and high frequencies and near spectral lines, reaching a roughly flat ("white") minimum in the band around 80 to 300 Hz.
We can "whiten" the data (dividing it by the noise amplitude spectrum, in the fourier domain), suppressing the extra noise at low frequencies and at the spectral lines, to better see the weak signals in the most sensitive band.
Whitening is always one of the first steps in astrophysical data analysis (searches, parameter estimation). Whitening requires no prior knowledge of spectral lines, etc; only the data are needed.
The resulting time series is no longer in units of strain; now in units of "sigmas" away from the mean.

For the function's arguments, strain is the original data from the file, the interpolated power spectrum density can be found by calling scipy.interpolate.interp1d on the power spectrum density you found earlier.  Call the whitening function on H1, L1 and the theoretical waveform.  Plot the results.

In [ ]:
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    # whitening: transform to freq domain, divide by asd, then transform back, 
    # taking care to get normalization right.
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(interp_psd(freqs) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

Run a bandpass filter on the data, which selects the range of frequencies we're interested in.  The scipy.signal library contains a function butter() that will generate a Butterworth filter - a filter that has a "flat" passband, and rejects signals outside that band.  It also has another function filtfilt that allows you to apply the filter to the data.  Apply a bandpass filter between 20 and 300 Hz to all 3 data sets.

Finally, we need to shift the L1 data set by 7 ms and invert it due to the direction the gravity wave came from and the alignment of the two interferometers.  Numpy's roll function allows you to shift the values of an array down a given number of data points (in this case we want to shift it by 7 ms worth of data points).  Shift the data and plot it one last time, you should clearly see the signal from the gravity wave.

Now take the data and output it as a sound around the event.  You may need to amplify the signal some to be able to hear it.